# 📄 LLM Agents can Autonomously Exploit One-day Vulnerabilities
> arXiv 2024 | Richard Fang, Rohan Bindu, Akul Gupta, Qiusi Zhan, Daniel Kang (UIUC)
> [논문 링크](https://arxiv.org/abs/2404.08144)

---

## ✅ Day 1 – Context & Motivation
*(AI가 단순 코딩 보조를 넘어, 실제 해킹을 수행하는 자율 에이전트로 진화하다)*

### 📌 연구 배경과 동기
- **보안 패치 격차 (The Patching Gap)**
  - 매일 수많은 취약점(CVE)이 공개되지만, 이를 검증하고 패치할 보안 전문가는 턱없이 부족하다.
  - 기존 취약점 스캐너(Nessus, OpenVAS 등)는 취약점의 존재 가능성만 알릴 뿐, 실제 공격 가능 여부(Exploitability)를 검증하지 못해 오탐(False Positive)이 많다.
- **기존 자동화의 한계**
  - 메타스플로잇(Metasploit) 등의 기존 도구는 사전에 정의된 스크립트만 실행하므로, 웹 환경이 조금만 달라져도 실패한다.
  - 공격을 성공시키려면 상황에 맞춰 판단하고 도구를 응용하는 '지능'이 필요하다.
- **핵심 문제의식**
  - *"최신 LLM(GPT-4)에게 해킹 도구와 권한을 쥐어준다면, 사람의 개입 없이 스스로 취약점을 분석하고 공격에 성공할 수 있을까?"*

### 📌 핵심 아이디어
- **LLM as an Autonomous Agent**
  - LLM을 단순한 챗봇이 아닌, 도구(Tools)를 사용하는 에이전트로 정의한다.
  - 웹 브라우저(Playwright), 터미널, 파일 관리 도구 등을 LLM에 연동하여 직접 시스템과 상호작용하게 한다.
- **ReAct 프레임워크 적용**
  - **Reasoning(추론) + Acting(행동)** 루프를 통해, LLM이 스스로 공격 계획을 세우고(Plan), 실행하고(Act), 결과를 해석하여(Observe) 다음 행동을 수정하는 방식을 채택한다.
- **One-day 취약점 정보 활용**
  - 제로데이(Zero-day)가 아닌, 공개된 CVE 설명(Advisory)을 입력으로 제공하여 LLM이 이를 이해하고 실제 공격 코드를 작성할 수 있는지 테스트한다.

### 📌 주요 기여 (Contributions)
1. **높은 공격 성공률 입증**
  - 실제 존재하는 15개의 심각한(Critical) One-day 취약점을 대상으로 실험한 결과, GPT-4 에이전트는 **87%**의 성공률을 기록했다. (GPT-3.5는 0%)
2. **압도적인 비용 효율성**
  - 전문 모의해킹 컨설턴트가 수백 달러를 받을 작업을, AI 에이전트는 건당 평균 **$8.80 (약 12,000원)**에 수행했다.
3. **범용 에이전트 가능성 제시**
  - 특정 취약점에 특화된 튜닝 없이, 범용 LLM에 프롬프트와 도구만 제공해도 복잡한 해킹 시나리오를 수행할 수 있음을 증명했다.

### 📌 핵심 인사이트
- **진입 장벽의 붕괴:** 해킹 전문 지식이 없어도 강력한 LLM만 있으면 누구나 정교한 공격을 시도할 수 있는 시대가 왔다. (Script Kiddie → **Prompt Kiddie**)
- **공격의 자동화 및 규모화:** 공격자는 AI 에이전트를 대량으로 복제하여 24시간 쉬지 않고 취약점을 탐색하고 공격할 수 있다.

### 🧠 개인적 해석
- 기존 보안관제(SOC)나 침해대응(CERT)이 방어 자동화에 집중했다면, 이 연구는 **공격 자동화**가 이미 현실 단계에 왔음을 보여준다.
- 모의해킹(Pen-testing) 직무에서 '단순 취약점 검증' 업무는 AI로 빠르게 대체될 것으로 보이며, 보안 전문가는 AI가 찾지 못하는 비즈니스 로직 취약점이나 AI 에이전트를 관리/검증하는 역할로 이동해야 할 것이다.
- 방어자 입장에서는 "사람이 공격한다"는 전제하에 세운 방어 주기를 "AI가 공격한다"는 전제로 수정하여 훨씬 더 민첩하게 대응해야 한다.

## ✅ Day 2 – Architecture & System Design
*(LLM 에이전트가 해킹 도구와 상호작용하는 구조)*

### 📌 1️⃣ 개요 (Overview)
> **핵심 목표:** CVE 설명서(텍스트)를 입력받아, 실제 취약한 시스템(웹/서버)과 상호작용하며 익스플로잇을 성공시키는 **자율 루프(Autonomous Loop)** 구축

### 🏗️ 2️⃣ 전체 구조 (Framework)

| 모듈/계층 | 구성요소 | 역할 |
|:---:|:---:|:---|
| **Controller** | **GPT-4 Agent** | 공격의 '두뇌' 역할. 상황을 판단하고 다음에 실행할 명령을 결정. |
| **Interface** | **Tool Manager** | LLM의 자연어 명령을 실제 시스템 명령(Shell/Code)으로 변환 및 실행. |
| **Environment** | **Target System** | 실제 취약점이 존재하는 격리된 샌드박스 환경 (Docker Container). |
| **Knowledge** | **CVE Description** | NVD(국가취약점데이터베이스) 등에서 가져온 취약점 설명 텍스트. |

### 🧩 3️⃣ 핵심 구성요소 세부 설명
#### (1) The Agent (LLM Core)
- GPT-4 (gpt-4-1106-preview) 모델을 베이스로 사용.
- 약 1,000토큰 분량의 **System Prompt**가 주입되어 있음 ("너는 전문 모의해킹 테스터이며, 목표는 이 시스템의 권한을 획득하는 것이다...").
- 이전의 모든 행동(History)을 기억하며 다음 행동을 추론함.

#### (2) Tools (무기)
- 에이전트가 사용할 수 있는 도구를 제한적이지만 강력하게 정의함.
  1.  **Terminal:** 쉘 명령어 실행 (`ls`, `curl`, `python`, `pip install` 등).
  2.  **File Editor:** 공격 코드(Python script) 작성 및 수정.
  3.  **Web Browser (Playwright):** 웹페이지 방문, HTML 분석, 버튼 클릭 등 (XSS/SQLi 공격 시 활용).
  4.  **Code Interpreter:** 파이썬 코드 실행 및 결과 확인.

### 🔁 4️⃣ 데이터 흐름 (Workflow)
1.  **Input:** CVE 설명 텍스트 ("Apache Log4j 취약점은 헤더 처리에 문제가 있어...")
2.  **Reasoning:** Agent가 설명을 읽고 "먼저 버전 확인을 위해 웹페이지 헤더를 스캔해야겠다"고 생각.
3.  **Action:** `curl -I http://target-ip` 명령어 실행 요청.
4.  **Observation:** 터미널 출력 결과(HTTP 200 OK...)를 Agent가 다시 읽음.
5.  **Loop:** 취약점을 찾을 때까지 2~4 과정 반복.
6.  **Output:** 공격 성공(Flag 획득) 또는 실패 선언.

### 🧠 개인적 해석
- 구조 자체는 복잡한 신경망 아키텍처가 아니라, **LLM에 '손발(Tools)'을 달아준 형태**이다.
- 핵심은 모델 자체의 성능(GPT-4)과, 이를 뒷받침하는 **환경 구성(Environment Setup)**에 있다.
- LangChain 같은 프레임워크와 유사하지만, 보안 공격에 특화된 툴셋(Terminal, Playwright)을 직접 쥐어줬다는 점이 차별점이다.

## ✅ Day 3 – Method & Mechanism
*(ReAct Loop의 수학적 모델링과 에이전트 구동 원리)*

### 📌 1️⃣ 핵심 메커니즘: ReAct Agent Framework
본 연구의 핵심은 LLM을 정적인 질의응답 모델이 아닌, 동적인 환경(Environment)과 상호작용하는 에이전트로 모델링한 것입니다.
이를 위해 **ReAct (Reasoning + Acting)** 프레임워크를 사용하여 에이전트의 행동을 $t$ 시점에서의 상태 전이(State Transition)로 정의합니다.

에이전트의 정책(Policy) $\pi$는 현재의 관측값(Observation) $O_t$와 이전의 히스토리 $H_t$를 기반으로 행동(Action) $A_t$를 결정합니다.

$$
A_t \sim \pi(A_t | O_t, H_t)
$$

여기서 $H_t$는 초기 프롬프트 $P$, 그리고 이전 시점까지의 모든 행동과 관측의 쌍으로 구성됩니다.

$$
H_t = \{P, (A_0, O_0), (A_1, O_1), \dots, (A_{t-1}, O_{t-1})\}
$$

### 🧪 2️⃣ 상세 프로세스 (The Exploitation Loop)
에이전트는 취약점 익스플로잇을 위해 다음과 같은 반복적인 루프를 수행합니다.

| 단계 | 기호 | 설명 | 구체적 예시 |
|:---:|:---:|:---|:---|
| **Step 1: Observation** | $O_t$ | 환경으로부터 현재 상태를 관측 | 터미널 출력: `HTTP 403 Forbidden` |
| **Step 2: Reasoning** | $R_t$ | 관측된 정보를 바탕으로 원인을 분석하고 계획 수립 (Chain-of-Thought) | *"403 에러는 권한 부족이다. 쿠키 탈취가 필요해 보인다."* |
| **Step 3: Action** | $A_t$ | 도구(Tool) 함수를 호출하여 환경에 영향 행사 | `Action: run_script('exploit.py')` |
| **Step 4: Update** | $H_{t+1}$ | 히스토리에 현재 행동과 결과를 추가하고 $t \leftarrow t+1$ | 컨텍스트 윈도우 업데이트 |



### 💬 3️⃣ 주요 테크닉 (Key Techniques)

#### (1) CVE-to-Prompt Transformation
입력으로 주어진 CVE 설명 $D_{cve}$를 에이전트가 이해할 수 있는 프롬프트 $P$로 변환하는 과정입니다.
단순 텍스트가 아니라, **"Goal(목표)"** 형태로 재정의하여 에이전트의 목적 지향성을 강화합니다.

$$
P = f_{prompt}(D_{cve}) + I_{system}
$$

* $D_{cve}$: *"Log4j vulnerability allows RCE..."*
* $I_{system}$: *"You are a penetration tester. Your goal is to..."* (System Instruction)

#### (2) Tool Abstraction Layer
LLM이 복잡한 운영체제 명령어를 직접 모두 알 필요 없이, 추상화된 도구 집합 $\mathcal{T}$를 제공하여 행동 공간(Action Space)을 효율적으로 제한합니다.

$$
A_t \in \mathcal{T} = \{ \text{Terminal}, \text{CodeEditor}, \text{WebBrowser}, \text{PyInterpreter} \}
$$

* 이러한 추상화는 모델이 문법 오류(Syntax Error)를 범할 확률 $P(Error)$를 낮추고, 논리적 추론에 집중하게 돕습니다.

#### (3) Self-Correction (자가 수정)
에이전트가 행동 $A_t$를 수행했으나 실패한 관측 $O_{fail}$을 얻었을 때, 이를 문맥 $H_{t+1}$에 포함시켜 다음 추론 $R_{t+1}$에서 같은 실수를 반복하지 않도록 합니다.

$$
\text{If } O_t = \text{Error}, \quad \text{then } R_{t+1} = \text{Analyze}(O_t) \rightarrow \text{New Plan}
$$

### 🧠 개인적 해석
- **수학적 최적화의 부재:** 기존 강화학습(RL)처럼 보상 함수(Reward Function) $R(s, a)$를 최대화하는 구조가 아니라, LLM의 사전 학습된 지식(Pre-trained Knowledge)에 의존하는 **In-context Learning** 방식이다.
- **탐색 공간의 문제:** 해킹은 가능한 행동의 공간(Search Space)이 무한대에 가깝다. ReAct 방식은 이 공간을 효과적으로 탐색하지만, $t$가 길어질수록(스텝이 많아질수록) 컨텍스트 제한으로 인해 초기 목표를 잊거나 환각(Hallucination)을 일으킬 위험이 존재한다.
- **실무적 시사점:** 코드를 짤 줄 모르는 사람도 논리 설계만 가능하다면, 이 $R_t$ (Reasoning) 파트만 잘 유도하여 고성능 해킹 도구를 만들 수 있다는 점이 가장 큰 위협 요인이다.

## ✅ Day 4 – Experiments & Evaluation
*(압도적인 GPT-4의 성능과 비용 효율성 검증)*

### 📌 1️⃣ 실험 환경 (Setup)
저자들은 실제 공격 시나리오를 재현하기 위해 **15개의 실제 리얼월드 취약점(Real-world CVEs)**으로 구성된 벤치마크를 구축했습니다.

- **Target Dataset:** $N=15$개의 One-day 취약점.
  - 웹 프레임워크, 컨테이너 관리 도구 등 다양함 (예: Log4j, Iris XSS, Hertzbeat RCE 등).
  - 모든 환경은 Docker Container로 격리되어 구축됨.
- **Agent Configuration:**
  - **Base Model:** GPT-4 (gpt-4-1106-preview), GPT-3.5-Turbo, LLaMA-2-70B-Chat 등.
  - **Input:** NVD(National Vulnerability Database)의 CVE 설명 텍스트(Description)만 제공.
- **Evaluation Metrics:**
  1.  **Success Rate ($SR$):** 에이전트가 실제로 시스템의 제어권(Flag)을 탈취했는지 여부.
  2.  **Cost ($C$):** 공격 성공까지 소모된 토큰 비용 (API Cost).

### 📊 2️⃣ 주요 결과 (Main Results)

가장 놀라운 점은 최상위 모델(GPT-4)과 하위 모델 간의 **'성능 격차(Capability Gap)'**입니다.

| Model Family | Model Name | Success Rate ($SR$) | Cost per Exploit ($C$) |
|:---|:---|:---:|:---:|
| **Proprietary** | **GPT-4 (Proposed)** | **$87\%$** (13/15) | **$\$8.80$** |
| **Proprietary** | GPT-3.5 Turbo | $0\%$ | - |
| **Open Source** | LLaMA-2 70B Chat | $0\%$ | - |
| **Open Source** | Mixtral-8x7B | $0\%$ | - |
| **Scanner** | ZAP / OpenVAS | $0\%$ | - |

> 💡 **결과 요약:** 오직 **GPT-4만이 유의미한 해킹 능력**을 보여주었습니다. GPT-3.5나 오픈소스 모델들은 복잡한 추론(Reasoning)과 도구 사용(Tool Use) 과정에서 논리적 오류를 범해 실패했습니다.



### 🔍 3️⃣ 분석 및 토의 (Detailed Analysis)

#### (1) Ablation Study: CVE 설명의 중요성
저자들은 에이전트에게 CVE 설명($D_{cve}$)을 주지 않았을 때의 성능 변화를 측정했습니다.
- **With Description:** $SR = 87\%$
- **Without Description:** $SR = 33\%$
- **해석:** 에이전트는 단순히 도구를 무작위로 실행하는 것이 아니라, **취약점 문서를 '읽고 이해하여'** 공격 전략을 수립한다는 것을 증명합니다 ($87\% - 33\% = 54\%p$의 성능 차이 발생).

#### (2) Cost Efficiency (비용 효율성)
인간 전문가와 AI 에이전트의 비용을 비교했을 때, AI의 경제성이 압도적입니다.
- **Human Expert:** 시간당 약 $\$100$ 이상 소요, 건당 수 시간 분석 필요.
- **GPT-4 Agent:** 건당 평균 **$\$8.80$ (약 12,000원)** 소요.
- 이는 공격자 입장에서 해킹 비용이 **$\approx \frac{1}{10}$ 이하**로 급감함을 의미합니다.

#### (3) Failure Analysis (실패 원인)
GPT-4가 실패한 2건(13/15 성공)의 경우, 웹 사이트의 HTML 구조가 너무 복잡해서 에이전트가 네비게이션에 실패하거나(Playwright 이슈), 컨텍스트 윈도우 제한으로 인해 초기 계획을 잊어버린 경우였습니다.

### 🧠 개인적 해석
- **모델의 지능 임계점(Threshold of Intelligence):** 해킹과 같은 복합적인 작업을 수행하려면 특정 수준 이상의 추론 능력이 필수적이다. 현재로서는 GPT-4급만이 이 '임계점'을 넘은 것으로 보인다.
- **방어의 비대칭성 심화:** 공격자는 단돈 만 원($\$8.80$)으로 고도의 해킹 시도를 자동화할 수 있다. 반면 방어자는 이를 막기 위해 값비싼 보안 장비와 인력을 유지해야 하므로, **공격의 가성비가 너무 좋아지는(Cost Asymmetry)** 심각한 상황이 도래했다.
- **실무 적용:** SK 쉴더스 관제/모의해킹 업무 시, "단순히 알려진 CVE를 막았는가?"를 넘어 "AI가 문서를 읽고 변형 공격을 시도할 때도 안전한가?"를 테스트하는 새로운 방법론이 필요해 보인다.

## ✅ Day 5 – Conclusion & Future Work
*(연구 총정리 및 사이버 보안의 미래에 대한 제언)*

### 📌 1️⃣ 연구 요약 (Summary of Contributions)
본 논문은 **"LLM이 스스로 해킹할 수 있는가?"**라는 질문에 대해, GPT-4 기반의 자율 에이전트를 구축하여 **실제 One-day 취약점의 87%를 익스플로잇**하는 데 성공함으로써 긍정적인 답을 내놓았습니다.

- **핵심 성과:**
  1.  **자율성(Autonomy):** 사람의 개입 없이 CVE 문서를 읽고 공격 계획을 수립 및 실행.
  2.  **범용성(Generalization):** 특정 취약점에 과적합(Overfitting)되지 않고, 다양한 유형의 웹/시스템 취약점에 대응 가능.
  3.  **경제성(Economy):** 인간 전문가 대비 약 2.8배 저렴한 비용으로 공격 수행 가능.

> 💡 **한 줄 요약:** "이제 고성능 LLM만 있다면, 누구나 저렴한 비용으로 강력한 사이버 공격을 감행할 수 있는 **'AI 해킹의 대중화'** 시대가 열렸다."

### ⚠️ 2️⃣ 한계점 (Limitations)
1.  **단일 모델 의존성:** 오직 **GPT-4**만이 성공적인 결과를 보여주었으며, 오픈소스 모델(LLaMA-2, Mixtral)이나 구형 모델(GPT-3.5)은 복잡한 추론을 감당하지 못해 실패했습니다.
2.  **제한된 환경:** 실험은 격리된 샌드박스(Docker) 내의 단일 취약점에 집중되었습니다. 실제 기업망처럼 복잡한 네트워크를 이동(Pivoting)하거나, 사회공학(Social Engineering) 기법을 섞는 복합 공격은 검증되지 않았습니다.
3.  **단순 에이전트 구조:** 현재의 에이전트는 상태 관리가 단순하여(Stateless에 가까움), 매우 긴 호흡이 필요한 APT(지능형 지속 위협) 공격 시나리오에서는 문맥을 잃을 가능성이 큽니다.

### 🚀 3️⃣ 향후 연구 방향 (Future Work)
- **Improving Open Source Agents:** GPT-4에 의존하지 않고, 보안 도메인에 특화된(Fine-tuned) 소형 LLM을 개발하여 에이전트 성능을 높이는 연구.
- **Defensive Agents (AI 방어자):** 공격뿐만 아니라, 취약점을 발견하고 **스스로 패치(Auto-patching)**하는 방어용 에이전트 개발.
- **Agent Security:** 반대로, 공격자 에이전트에게 잘못된 정보를 주입(Prompt Injection)하여 공격을 무력화하는 기법 연구.

### 🧠 5️⃣ 종합 평가 및 회고 (Personal Review)
- **실무 적용성:** ⭐⭐⭐⭐⭐ (매우 높음)
  - 이미 공격 툴은 완성 단계에 가까우며, 당장 모의해킹 예비 진단 도구로 투입 가능함.
- **추천 대상:** 모의해킹 컨설턴트, 보안 아키텍트, 그리고 **'AI 시대에 내가 설 자리가 어디인가'**를 고민하는 모든 보안 입문자.
- **한 줄 평:** *"스크립트 키디(Script Kiddie)의 시대는 가고, 이제 **프롬프트 키디(Prompt Kiddie)**의 시대가 왔다. 방어자는 더 이상 사람의 속도에 맞춰서는 안 된다."*

---
**[Final Thought]**
SK 쉴더스 교육생으로서 이 논문을 통해 느낀 점은, 단순히 해킹 툴 사용법(Metasploit, Burp Suite)을 익히는 것보다 **'그 툴을 언제, 왜 써야 하는지'를 판단하는 설계 능력**이 훨씬 중요해졌다는 것이다. 기술적인 실행은 이제 AI가 나보다 더 잘할 수도 있다. 나는 AI에게 정확한 지시를 내리고 결과를 검증할 수 있는 **'보안 아키텍트'**가 되어야겠다.